<img src="img/Asesoftware_logo.png" width="200" height="100">
<center>
    <h1>Análisis Postmortem</h1>
    <h3>Autores: <i>Laura Benavides, Rodolfo de la Rosa, Álvaro Valbuena</i></h3>
    <h4><i>lbenavides@asesoftware.com, rdelarosap@asesoftware.com, avalbuena@asesoftware.com</i></h4>
    <h3>Área de Innovación</h3>
    <br>
</center>


El procesamiento de lenguaje natural (o NLP por sus siglas en inglés) es una rama de conocimiento de la Inteligencia Artificial y la lingüística que, esencialmente, tiene como objetivo que los computadores "comprendan" el lenguaje natural para así poder desarrollar tareas útiles.

NLP involucra el análisis de:

1. **Sintaxis**: identificar el rol sintáctico de cada palabra en la oración. E.g. dulces es objeto directo en: Yo compraré dulces.

2. **Semántica**: determinar el significado de las palabras o frases. E.g. "El ingeniero fue al cliente". El cliente se refiere a un lugar.

3. **Pragmática**: establecer como el contexto comunicativo afecta el significado. E.g. "El tomó vino en la fiesta. Era rojo". Rojo se refiere al vino.

Los dos desafíos más comunes en NLP son: Ambigüedad y combinar contexto con conocimiento previo.

Para el análisis de los documentos post-mortem se van a seguir el típico pipeline en NLP:

1. *Tokenization*
2. *Lemmatization or Stemming*
3. *Sentence boundary detection*
4. *POS tagging*
5. *Parsing*
6. *NER*
7. *Coreference resolution*
8. *Infomation Extraction*

Durante la implementación de este proyecto se verán algunos problemas que no permitieron completar todos los pasos anteriores y que hicieron tomar otras aproximaciones.


### Librerías necesarias

#### Descargar paquetes

In [ ]:
!conda install -c conda-forge spacy -y

In [ ]:
!python -m spacy download es_core_news_md

In [ ]:
!conda install -c anaconda pandas -y

In [ ]:
!conda install -c anaconda xlrd -y

In [ ]:
!conda install -c conda-forge textacy -y

In [ ]:
!conda install -c anaconda nltk -y

## 1. *Tokenization*

El primer paso en el procesamiento de lenguaje natural es poder separar una sequencia de caracteres en una sequencia de tokens. Un token se define como una secuencia de caracteres con significado (normalmente son palabras, sin embargo pueden ser n-grams). A este proceso se le llama **_tokenization_**.

Un ejemplo es usar el espacio para separar los tokens: 

### La inteligencia es la habilidad de adaptarse a los cambios.

Se convierte:

### | La | inteligencia | es | la | habilidad | de | adaptarse | a | los | cambios. |

Las expresiones regulares son usadas para realizar este proceso. A continuación hay dos ejemplos de *tokenization* por medio de dos expresiones regulares diferentes:

In [ ]:
import nltk

from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import TreebankWordTokenizer

frase = 'La inteligencia es*la habilidad de adaptarse a los cambios...'

# Tokenizador por expresiones regulares con espacios en blanco
regexp_tokenizer = RegexpTokenizer(r'\w+')

# Tokenizador por expresiones regulares para tokenizar texto de acuerdo al Penn Treebank.
treebank_tokenizer = TreebankWordTokenizer()

print(regexp_tokenizer.tokenize(frase))
print(treebank_tokenizer.tokenize(frase))

## 2. _Lemmatization or Stemming_

El objetivo de este paso es normalizar o cannonizar los tokens a una forma reducida. Para esto, hay dos técnicas diferentes:

- **Stemming:**

En _stemming_ , o derivación regresiva, lo que se busca es obtener la raíz o *stem* de la palabra, quitando afijos mediante un algoritmo. Muchas veces el resultado no son palabras reales: computadora y computación tendrían como *stem* comput.

En español, el algoritmo que mejor funciona es el Snowball Stemmer: https://snowballstem.org/algorithms/spanish/stemmer.html

In [ ]:
# Stemming con NLTK
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('spanish')

def normalize(tokens):
    normalized_tokens = list()
    for token in tokens:
        # Primero se pone todo el texto en minúscula
        normalized = token.lower()
        # Se llama el stem del NLTK
        normalized = stemmer.stem(normalized)
        normalized_tokens.append(normalized)
    return normalized_tokens


frase_normalizada1 = normalize(regexp_tokenizer.tokenize(frase)) 

print('Frase:',frase)
print('Stems:',frase_normalizada1)

- **Lemmatization:**

Es el proceso de agrupar las diferentes formas de una palabra a una forma base o *lemma*. Este proceso usa  el POS tagging de las palabras, el contexto de la frase y hasta el contexto del corpus para generar el lemma de una palabra. 

Este proceso busca retornar por cada palabra una forma base tal cual se encontraría en un diccionario como Wordnet.

Es un proceso más lingüíticamente adecuado que el *stemming* pero depende de la precisión del contexto y del diccionario. Este proceso es más lento y más costoso que el *stemming*

In [ ]:
import pandas as pd
import spacy

# Importar vocabulario de Spacy, removiendo del pipeline el NER
nlp = spacy.load('es_core_news_md', disable=['ner'])

In [ ]:
word = []
lemma = []
punct = []
space = []
shape = []

text_nlp=nlp(frase)
for token in text_nlp:
    word.append(token.text)
    lemma.append(token.lemma_)
    punct.append(token.is_punct)
    space.append(token.is_space)
    shape.append(token.shape_)

    
pd_lemmas = pd.DataFrame({'Word':word, 'Lemma':lemma, 'Punct':punct, 'Space':space, 'Shape':shape})
# Muestra la palabra, el lemma, si es puntuación, si es espacio, y  la forma de la palabra
pd_lemmas

## 3. Sentence boundary detection



## 4. Part-Of-Speech (POS) tagging

Consiste en asignarle (o etiquetar) a cada una de las palabras de un texto de su categoría gramatical (Nombre, pronombre, adjetivo, verbo...) de acuerdo a su contexto y su significado.

Los dos POS-tagger más conocidos para español son:

1. StanfordPOSTagger (NLTK)
2. SpaCy



_**<font color=red>NOTA:</font>** Para que el POS-tagger de Stanford funcione correctamente es necesario descargar los modelos completos de https://nlp.stanford.edu/software/tagger.shtml#Download y descomprimirlos en la ruta de las variables `tagger`y `jar` de la siguiente celda_

In [ ]:
# POS tagging de Stanford
from nltk.tag import StanfordPOSTagger

tagger = "sources/models/spanish.tagger"
jar = "sources/stanford-postagger.jar"

tagger = StanfordPOSTagger(tagger,jar)
tags = tagger.tag(regexp_tokenizer.tokenize(frase.lower()))
for tag in tags:
    print(tag)

In [ ]:
# POS tagging de Spacy
word = []
lemma = []
shape = []
pos = []
istop = []

for token in text_nlp:
    word.append(token.text)
    lemma.append(token.lemma_)
    shape.append(token.shape_)
    pos.append(token.pos_)
    istop.append(token.is_stop)
    
pd_pos = pd.DataFrame({'Word':word, 'Lemma':lemma, 'Shape':shape, 'POS':pos, 'is_stop':istop})
pd_pos

## 5. Parsing

Existen varios tipos de parsing: *constituency, dependency y semantic.*

Para este proyecto se decidió solo concentrarse en el *dependency parsing*.

Este proceso analiza la estructura gramatical de una frase, estableciendo relaciones entre palabras "origen" y palabras que modifican esas palabra origen (o dependientes). 

Estas relaciones gramaticales son representadas como arcos entre las palabras de la oración. La dirección de los arcos indica la relación _desde_ la palabra origen _hacia_ la palabra dependiente.

Esto genera un grafo dirigido por cada oración.


In [ ]:
spacy.displacy.render(text_nlp)

## 6. Named Entity Recognition (NER)

Es el proceso de encontrar y clasificar nombres/entidades en el texto. Las clasificaciones varían según la herramienta que se use. Ejemplos de categorias pueden ser:

1. Personas
2. Locaciones
3. Organizaciones
4. Miscelánea

Este proceso corresponde a usar un clasificador ya pre-entrenado con las palabras del texto que se está analizando. Estos clasificadores suelen ser *sequence-labeling models*. Los métodos más sofisticados usan RNN (redes neuronales recurrentes) bidireccionales.


## 7. Coreference Resolution

Es el proceso mediante el cuál se encuentran todas las expresiones que se refieren a la misma entidad en el texto.

e.g. "**Obama** visitó la ciudad. **El presidente** habló sobre el cambio climático. **Él** mencionó los problemas ambientales que vendrán en los próximos años."

Las palabras en negrilla se refieren a la misma entidad: Obama.

Para implementar este paso, se usan modelos supervisados de machine learning o en algunos casos heurísticas.


## 8. Information Extraction

Una de las tareas típicas en IE corresponde a identificar entidades, encontrar relaciones entre estas entidades y encontrar eventos. Las relaciones se pueden presentar en formas como: contiene, es, regula, causa, interactua, está compuesto...

Toda la información recolectada hasta este paso es usada para transformar el texto escrito en lenguaje natural en información estructurada como por ejemplo una base de datos.

***
***

<h1><center>NLP con archivos post-mortem</center></h1>

En Asesoftware existen archivos con la información post-mortem de los proyectos, estos son los documentos de cierre de proyecto. En este documento, existe una sección donde se agrupan las lecciones aprendidas en el desarrollo del proyecto.

Esta información está agrupada en un archivo excel donde se encuentra la siguiente información por proyecto: tipo de proyecto, contexto, lección aprendida, entre otras caracteristicas.

El siguiente script obtiene en un dataframe de Pandas la información de contexto y lección aprendida.

In [ ]:
# Función para concatenar dos celdas.
def concat_text(pdSeries):
    pdSeries = pdSeries.str.rstrip('.')
    return pdSeries.str.cat(sep='. ')

In [ ]:
# Archivo con las lecciones aprendidas
nombre_archivo = "data/REPOSITORIO_LECCIONES APRENDIDAS.xlsx"

# Lectura del archivo
data = pd.read_excel(nombre_archivo, encoding='latin-1', keep_default_na= False, na_values=[""])

# Se concatena el contenido de las columnas "CONTEXTO"  y "LECCIONES APRENDIDAS"
data["CONTEX_LECC"] = [concat_text(i[1]) for i in data[['CONTEXTO', 'LECCIONES APRENDIDAS']].iterrows()]

# Se quitan espacios y enter de la columna CONTEXT_LECC
data.CONTEX_LECC = data.CONTEX_LECC.str.replace('\n', ' ').replace('\s+', ' ')
data.head()

En la columna `CONTEX_LECC` están concatenadas las oraciones de contexto y lecciones, y `cont_lecc` (en la siguiente celda) es el texto que se va a analizar con el pipeline de NLP.

In [ ]:
cont_lecc = concat_text(data["CONTEX_LECC"])

En Spacy, el modelo entrenado en español incluye en su _pipeline:_

- Tokenizer
- Lemmatizer
- POS-tagger
- NER

Sin embargo, por los resultados obtenidos en pruebas concluimos que el NER de Spacy necesita ser mejorado para poder usarlo con confianza en este proceso de extracción de información.

En la siguiente celda se extraen los lemmas, el POS-tagging y el dependency parsing de la columna `CONTEX_LECC`, junto con una lista de _dependientes_ (children) de la palabra en el árbol del dependency parsing

In [ ]:
# Listas correspondientes a cada componente
word = []
lemma = []
shape = []
pos = []
istop = []
dep = []
head = []
children = []

nlp_text=nlp(cont_lecc)

# Recorer la lista de tokens del texto y obtener su información respecto a lemma, forma, POStagging, si es stopword, 
# dependency parsing, raiz y ramas
for token in nlp_text:
    word.append(token.text)
    lemma.append(token.lemma_)
    shape.append(token.shape_)
    pos.append(token.pos_)
    istop.append(token.is_stop)
    dep.append(token.dep_)
    head.append(token.head.text)
    children.append([child for child in token.children])

    
results = pd.DataFrame({'Word':word, 'Lemma':lemma, 'POS':pos, 'DEP':dep, 'head':head,
                             'children':children, 'Shape':shape, 'is_stop':istop})
results.head()

El objetivo, una vez obtenidas las categorías gramaticales (POS) y las dependencias es extraer entidades y sus relaciones.

Esto se puede lograr mediante:
* Reglas
* Modelos de machine learning no supervisados (clustering o neural networks)

Para extraer reglas, se analizaron las gráficas de dependency parsing y se probó con las siguientes relaciones.

In [ ]:
info = []

for possible_subject in nlp_text:
    # si el POStagging del HEAD de la palabra es VERB (verbo) y su dependency parsing es nsubj (sujeto nominal)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.dep_ == 'nsubj' :
        children = []
        for child in possible_subject.children:
            # si las ramas son nmod (modificador nominal) y no es espacio
             if child.dep_ in ('nmod') and child.pos_ != 'SPACE': 
                children.append(child)
            
        if children:
            info.append((possible_subject.head.lemma_.lower(),possible_subject.lemma_.lower(),children))
result = pd.DataFrame(info, columns = ['Head' , 'Word', 'Children'])
result.head()

In [ ]:
info = []
for possible_subject in nlp_text:
    # Si el POStagging del HEAD de la palabra es VERB y el POStagging de la palabra es un sustantivo (PROPN y NOUN) y
    # su dependency parsing es sujeto nominal (nsubj)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.pos_ in ('PROPN','NOUN') and possible_subject.dep_=='nsubj':
        info.append((possible_subject.head,possible_subject,possible_subject.lemma_))
        
result_subj = pd.DataFrame(info, columns = ['Head','Word', 'Lemma'])
result_subj.head(n=10)

In [ ]:
info = []
for possible_subject in nlp_text:
    # Si el POStagging del HEAD de la palabra es VERB y el POS de la palabra es un sustantivo (PROPN y NOUN) y
    # su dependency parsing es sujeto nominal (nsubj)
    if possible_subject.head.pos_ == 'VERB' and possible_subject.pos_ in ('PROPN','NOUN') and possible_subject.dep_=='nsubj':
        children = []
        for child in possible_subject.children:
            # Solo agregar si no es identificado como espacio
            if child.pos_ != 'SPACE':
                children.append(child)
            
        if children:
            info.append((possible_subject.head.lemma_,possible_subject,possible_subject.lemma_.lower(),children))
            
result_subj1 = pd.DataFrame(info, columns = ['Head' , 'Word', 'Lemma', 'Children'])
result_subj1.head(n=10)


In [ ]:
info = []
for possible_subject in nlp_text:
    # Si el POStagging de la palabra es VERB
    if possible_subject.pos_ == 'VERB' :
        children = []
        for child in possible_subject.children:
            # si las ramas son nsubj (sujeto nominal) y no es espacio
            if child.dep_=='nsubj' and child.pos_ != 'SPACE':
                children.append(child)
            
        if children:
            info.append((possible_subject.head.lemma_,possible_subject,children))
            
result_subj2 = pd.DataFrame(info, columns = ['Head' , 'Word', 'Children'])
result_subj2.head(n=10)

## Reconocimiento de entidades mediante reglas:

In [ ]:
from __future__ import unicode_literals
import textacy
from collections import defaultdict

###
# Patrón para extraer información de un texto basado en reglas (utilizando expresiones regulares) utilizando los resultados del postagging.
###

# se crea la siguiente regla:
patron = r'<PROPN>+ (<PUNCT|CCONJ> <PUNCT|CCONJ>? <PROPN>+)*'

# la regla se utiliza con la libreria textacy para encontrar las entidades de las leciones aprendidas.
param = []
i = 0
while i < len(data["CONTEX_LECC"]):
    lists_ = []
    sent = nlp(data["CONTEX_LECC"].iloc[i])
    doc = textacy.make_spacy_doc(sent, lang='es_core_news_md')
    lists_ = textacy.extract.pos_regex_matches(doc, patron)
    for item in lists_:
        if len(item) != 0:
            param.append(item.text.lower())
    i +=1


# Resultados obtenidos
j=0
aux = defaultdict(list)
for index, item in enumerate(param):
    aux[item].append(index)

result = {item: len(indexs) for item, indexs in aux.items() if len(indexs) >= 1}
entity_list = list(result.keys())
entity_list.sort()

In [ ]:
# Para visualizar contenido de 'Entidades'
entity_list

Se observa que con esta regla se obtienen algunas entidades que estan referenciadas en el texto de forma diferente.

Por ejemplo: Davivienda, Banco Davivienda, están haciendo referencia a la misma entidad.

### Combinación de la regla de entidades con la regla de relaciones en la siguiente celda

In [ ]:
info = []

for possible_subject in nlp_text:
    
    # Si la palabra se encuentra en la lista de entidades
    if str(possible_subject).lower() in entity_list:
        
        # si el POStagging del HEAD de la palabra es VERB y su dependency parsing es nsubj (sujeto nominal)
        if possible_subject.head.pos_ == 'VERB' and possible_subject.dep_ == 'nsubj' :
            
            for child in possible_subject.children:
                # Si la palabra de la rama se encuentra en la lista de entidades            
                if str(child).lower() in entity_list:
                    
                    # si las ramas son nmod (modificador nominal) y no es espacio
                     if child.dep_ in ('nmod') and child.pos_ != 'SPACE':
                            info.append((possible_subject.text.lower(),possible_subject.head.lemma_.lower(),child.text.lower()))

result_ent = pd.DataFrame(info, columns = ['Word', 'Head', 'Children'])
result_ent.head()

In [ ]:
info = []
for possible_subject in nlp_text:
    # Si la palabra se encuentra en la lista de entidades
    if str(possible_subject).lower() in entity_list:
        # Si el POStagging del HEAD de la palabra es VERB y el POS de la palabra es un sustantivo (PROPN y NOUN)
        if possible_subject.head.pos_ == 'VERB' and possible_subject.pos_ in ('PROPN','NOUN'):

            for child in possible_subject.children:
                
                # Si la palabra de la rama se encuentra en la lista de entidades
                if str(child).lower() in entity_list:
                    
                    # Solo agregar si no es identificado como espacio
                    if child.pos_ != 'SPACE':
                        info.append((possible_subject.text.lower(),possible_subject.head.lemma_.lower(),child.text.lower()))

result_ent1 = pd.DataFrame(info, columns = ['Word', 'Head', 'Children'])
result_ent1.head()

### Palabras relacionadas con 'Asesoftware'

In [ ]:
search = 'asesoftware'
asw_1 = result_ent[(result_ent.Word.str.contains(search)) | (result_ent.Children.str.contains(search))]
asw_1

In [ ]:
asw_2= result_ent1[(result_ent1.Word.str.contains(search))| (result_ent1.Children.str.contains(search))]
asw_2

De las lecciones aprendidad al ejecutar las dierentes reglas se extrae información relevenate, por ejemplo de Asesoftware se dice lo siguiente:
- Asesoftware guía y realiza procesos de prueba.    

### Observaciones:

Durante el análisis de resultados, se observó que habían entidades "ambiguas", como _cliente_ o _proyecto_ y se consideró importante saber, por ejemplo, de qué cliente o tipo de proyecto se refería esta entidad.

Ya que dentro del archivo de Lecciones aprendidas está esta información concreta (las columnas 'CLIENTE' y 'TIPO PROYECTO'), se planteó reemplazar las palabras por el contenido de esas columnas. Sin embargo, esta modificación afecta el POStagging y el dependency parsing, por lo que se decidió dejar de lado esta aproximación.